# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries - general ones
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
import re
import pickle

In [2]:
# importing libraries for natural language processing
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [3]:
# importing libraries for machine learning
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.multioutput import MultiOutputClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report

In [4]:
# load data from database
engine = create_engine('sqlite:///Messages_ETL.db')
df = pd.read_sql_table('Messages_ETL', engine)
# X: Our input to be classified
X = df['message']
# Y: The goal of our classification
Y = df.iloc[:,4:] # Y are all the other columns except the first 4

In [5]:
print(Y.shape)
Y.columns

(26028, 35)


Index(['related', 'request', 'offer', 'aid_related', 'medical_help',
       'medical_products', 'search_and_rescue', 'security', 'military',
       'water', 'food', 'shelter', 'clothing', 'money', 'missing_people',
       'refugees', 'death', 'other_aid', 'infrastructure_related', 'transport',
       'buildings', 'electricity', 'tools', 'hospitals', 'shops',
       'aid_centers', 'other_infrastructure', 'weather_related', 'floods',
       'storm', 'fire', 'earthquake', 'cold', 'other_weather',
       'direct_report'],
      dtype='object')

### 2. Write a tokenization function to process your text data

In [6]:
def tokenize(text, url_place_holder_string = "urlplaceholder"):
    """
    Tokenize the text, replacing urls with a urlplaceholder
    
    Parameters
    ----------
    text : str
        The text to be tokenized
    urlplaceholder : str
        Optional keyword for replacing the urls 
      
    Returns
    -------
    clean_tokens : List of tokens extracted from the text

    """
    # Replacing all urls with placeholder
    
    # Regular expression for detecting URLs
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    detected_urls = re.findall(url_regex, text)
    
    # Replace each URL in text string with placeholder
    for url in detected_urls:
        #text = re.sub(url, url_place_holder_string, text) causes error because some detected urls contain brackets
        text = text.replace(url, url_place_holder_string)

    # Replacing all punctuation with space making sure no brackets are left
    text = re.sub(r"[^a-zA-Z0-9]+", " ", text)
    
    # Extractring the tokens from the text
    tokens = word_tokenize(text)
    
    # Removing stopwords
    tokens = [token for token in tokens if token not in stopwords.words('english')]
    
    # Using lemmatizer to remove inflection, endings, ...
    # Initiating lemmatizer
    lemmatizer = WordNetLemmatizer()
    
    # Iterate through each token, using list comprehension
    clean_tokens = [lemmatizer.lemmatize(token).lower().strip() for token in tokens]
    
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [7]:
pipeline = Pipeline([
    ('counts_vectorizer', CountVectorizer(tokenizer=tokenize)), 
    ('tfidf_transformer', TfidfTransformer()), 
    ('classifier', MultiOutputClassifier(RandomForestClassifier()))
])

In [8]:
# Pipeline with a different Classifier
pipeline2 = Pipeline([
    ('counts_vectorizer', CountVectorizer(tokenizer=tokenize)), 
    ('tfidf_transformer', TfidfTransformer()), 
    ('classifier', MultiOutputClassifier(GradientBoostingClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [9]:
# Splitting the data into train and test sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y)

In [10]:
# Training the pipeline
pipeline.fit(X_train, Y_train)

Pipeline(memory=None,
     steps=[('counts_vectorizer', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [ ]:
# Predict on X test data
Y_pred = pipeline.predict(X_test)

In [ ]:
# Report the f1 score, precision and recall for each category

for i in range(len(Y_test.columns)):
    print(f"Category: {Y_test.columns[i]}")
    print(classification_report(Y_test.iloc[:,i], Y_pred[:,i]))

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
# Get parameters
pipeline.get_params()

In [ ]:
# Main goal is classifying the messages, searching part of the classifier space, showing the principle by choosing two different
# parameters to optimize, using only two values each because otherwise it took forever
parameters ={
    'classifier__estimator__n_estimators' : [50, 100],
    'classifier__estimator__min_samples_split' : [2, 3]
}

# Creating grid search object, adding verbose to see that it is doing something
cv = GridSearchCV(pipeline, param_grid=parameters, verbose = 3)

In [ ]:
# Running the grid search
cv.fit(X_train, Y_train)

In [ ]:
# Print best parameters after doing grid search
print(cv.best_params_)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
# Getting new predictions with the result from the grid search
Y_cv_pred = cv.predict(Y_test)

# Report the f1 score, precision and recall for each category

for i in range(len(Y_test.columns)):
    print(f"Category: {Y_test.columns[i]}")
    print(classification_report(Y_test.iloc[:,i], Y_cv_pred[:,i]))

In [ ]:
# Using best estimator to define an optimised model
optimised_model = cv.best_estimator_
cv.best_estimator_

In [ ]:
# Using the optimised model for predicting new Ys
Y_pred_opt = optimised_model.predict(X_test)

In [ ]:
# Report the f1 score, precision and recall for each category

for i in range(len(Y_test.columns)):
    print(f"Category: {Y_test.columns[i]}")
    print(classification_report(Y_test.iloc[:,i], Y_pred_opt[:,i]))

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
# Training the pipeline with another pipeline containing XGBoost
pipeline2.fit(X_train, Y_train) 

In [ ]:
# Predict on X test data
Y_pred_2 = pipeline2.predict(X_test)

In [ ]:
# Report the f1 score, precision and recall for each category

for i in range(len(Y_test.columns)):
    print(f"Category: {Y_test.columns[i]}")
    print(classification_report(Y_test.iloc[:,i], Y_pred_2[:,i]))

### 9. Export your model as a pickle file

In [ ]:
with open('classifier.pkl', 'wb') as file:
    pickle.dump(optimised_model, file)

### 10. Use this notebook to complete `train_classifier.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.